In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
fpl_midfielders = pd.read_csv('../data/processed/epl_midfielder_predictions.csv')
sorare_midfielders = pd.read_csv('../sorare/sorare_data/predictions/sorare_midfielders_predictions.csv')

print(fpl_midfielders.columns)
print(sorare_midfielders.columns)

fpl_midfielders = fpl_midfielders.rename(columns={'first_name': 'First_Name', 'second_name': 'Last_Name', 'epl_predictions':'Predicted_Points'})
fpl_midfielders_filtered = fpl_midfielders[['id','First_Name','Last_Name', 'Predicted_Points']]

sorare_midfielders_filtered = sorare_midfielders[['First_Name','Last_Name', 'So_5_Scores_9', 'sorare_predictions']]

print(fpl_midfielders.shape)
print(sorare_midfielders.shape)

print("------------------------")

print(fpl_midfielders_filtered.shape)
print(sorare_midfielders_filtered.shape)

Index(['season_name', 'id', 'first_name', 'second_name', 'team',
       'epl_predictions'],
      dtype='object')
Index(['Display_Name', 'First_Name', 'Last_Name', 'Player_Number', 'Position',
       'Current_Club', 'So_5_Scores_9', 'sorare_xgb_predictions',
       'sorare_lgbm_predictions', 'sorare_elastic_predictions',
       'sorare_predictions'],
      dtype='object')
(921, 6)
(151, 11)
------------------------
(921, 4)
(151, 4)


In [3]:
midfielders = pd.merge(sorare_midfielders_filtered, fpl_midfielders_filtered, on=['First_Name', 'Last_Name'] )

print(midfielders.shape)

midfielders['Weighted_BOR3_Score'] = (midfielders['sorare_predictions'] * 0.75) + (midfielders['Predicted_Points'] * 0.25)

(456, 6)


In [4]:
midfielders = midfielders.sort_values('Weighted_BOR3_Score', ascending=False)

midfielders.to_csv('../data/predictions/weighted_bor3_midfielders.csv')
midfielders

,First_Name,Last_Name,So_5_Scores_9,sorare_predictions,id,Predicted_Points,Weighted_BOR3_Score
270,Kevin,De Bruyne,85.4,71.873509,345,82.14,74.440132
268,Kevin,De Bruyne,85.4,71.873509,345,69.73,71.337632
267,Kevin,De Bruyne,85.4,71.873509,345,65.82,70.360132
272,Kevin,De Bruyne,85.4,71.873509,345,63.34,69.740132
53,Christian,Eriksen,100.0,67.848911,370,73.82,69.341683
...,...,...,...,...,...,...,...
46,Carney,Chukwuemeka,0.0,1.662627,161,4.52,2.376970
253,Josh,Dasilva,0.0,0.963476,90,2.77,1.415107
44,Carney,Chukwuemeka,0.0,1.662627,161,0.64,1.406970
288,Lewis,Miley,0.0,1.120040,411,0.49,0.962530


In [5]:
y_true = midfielders['So_5_Scores_9'].to_numpy() 
y_pred = midfielders['Weighted_BOR3_Score'].to_numpy()

# Calculate r2
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
print(f'MSE: {mean_squared_error(y_true, y_pred)}')
print(f'RMSE: {rmse}')
r2 = r2_score(y_true, y_pred)
print(f'r2: {r2}')

MSE: 223.14827965379007
RMSE: 14.938148468059556
r2: 0.6437411793394356
